In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/3.vehicle'):
    shutil.rmtree('./file/3.vehicle')

os.makedirs('./file/3.vehicle')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import json
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/2.item'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/2.item/{file}',
                           header=0,
                           dtype=str).fillna('')

    input_ = input_[input_['Json_Compatibility'] != '{}'].reset_index(drop=True)
    
    input_['No'] = input_['No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                df_temp, json_, page = pd.DataFrame(), json.loads(input_.loc[a, 'Json_Compatibility']), 1
                while page <= json_['page']:
                    b = 0
                    while True:
                        b += 1

                        try:
                            resp = requests.post(f'''{json_['api']}?module_groups=PART_FINDER&referrer=VIEWITEM&offset={page*20-20}&module=COMPATIBILITY_TABLE''',
                                                 data=json.dumps(json_['data']),
                                                 headers=get_header(ua=False, **{'Content-Type': 'application/json; charset=utf-8'}),
                                                 proxies=get_proxy(),
                                                 timeout=(10, 10))
                
                            if resp.status_code == 200:
                                break
                        except KeyboardInterrupt:
                            break
                        except:
                            continue
                
                    # = = = = = = = = = = = = = = =
                
                    list_header = [cell['textSpans'][0]['text'].strip() for cell in resp.json()['modules']['COMPATIBILITY_TABLE']['paginatedTable']['header']['cells']]
                
                    # = = = = = = = = = = = = = = =
                
                    list_row = [[cell['textualDisplays'][0]['textSpans'][0]['text'].strip() if 'textualDisplays' in cell else cell['textSpans'][0]['text'].strip() for cell in row['cells']] for row in resp.json()['modules']['COMPATIBILITY_TABLE']['paginatedTable']['rows']]
                
                    # = = = = = = = = = = = = = = =
                
                    df_temp_temp = pd.DataFrame({'No': input_.loc[a, 'No'],
                                                 'Item Number': input_.loc[a, 'Item Number'],
                                                 'Json_Compatibility': input_.loc[a, 'Json_Compatibility'],
                                                 'Page': page,
                                                 'Row': [i+1 for i in range(len(list_row))]})
                
                    for i, row in zip(range(len(list_row)), list_row):
                        for j, header in zip(range(len(list_header)), list_header):
                            df_temp_temp.loc[i, header] = row[j]
                
                    df_temp = pd.concat([df_temp, df_temp_temp], ignore_index=True).fillna('')
                
                    # = = = = = = = = = = = = = = =

                    if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 100 == 0 and work.qsize() != 0:
                        clear_output()

                    print(f'''[尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} - {input_.loc[a, 'No']}.{input_.loc[a, 'Item Number']} > Page {page}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

                    # = = = = = = = = = = = = = = =
                
                    page += 1

                # = = = = = = = = = = = = = = =

                if df_temp.empty:
                    raise

                # = = = = = = = = = = = = = = =

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(5):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['No', 'Page', 'Row'],
                                                    ascending=[True, True, True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/3.vehicle/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-vehicle_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/3.vehicle'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/3.vehicle/{file}',
              f'''./file/3.vehicle/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

[尝试次数：2] - 100.00% - crawler_2 - 4303.375394632967 > Page 6
[剩余数量：100] - [当前时间：22:39:23]

[尝试次数：1] - 100.00% - crawler_2 - 4336.304287930364 > Page 3
[剩余数量：99] - [当前时间：22:39:23]

[尝试次数：1] - 100.00% - crawler_2 - 4347.395369962128 > Page 1
[剩余数量：99] - [当前时间：22:39:23]

[尝试次数：1] - 100.00% - crawler_2 - 4349.374920743272 > Page 2
[剩余数量：98] - [当前时间：22:39:24]

[尝试次数：1] - 100.00% - crawler_2 - 4332.394598818880 > Page 3
[剩余数量：97] - [当前时间：22:39:24]

[尝试次数：1] - 100.00% - crawler_2 - 4352.374036584680 > Page 1
[剩余数量：97] - [当前时间：22:39:24]

[尝试次数：1] - 100.00% - crawler_2 - 4341.393793623507 > Page 3
[剩余数量：97] - [当前时间：22:39:24]

[尝试次数：1] - 100.00% - crawler_2 - 4350.304455714533 > Page 1
[剩余数量：97] - [当前时间：22:39:24]

[尝试次数：1] - 100.00% - crawler_2 - 4348.374373329478 > Page 1
[剩余数量：97] - [当前时间：22:39:24]

[尝试次数：1] - 100.00% - crawler_2 - 4312.373828203946 > Page 10
[剩余数量：97] - [当前时间：22:39:25]

[尝试次数：1] - 100.00% - crawler_2 - 4351.394647664645 > Page 1
[剩余数量：96] - [当前时间：22:39:25]

[尝试次数：1] - 100.00% 

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 4040.76it/s]

Done ~
